In [1]:
import trr
import numpy as np
import timer
import MDAnalysis as mda
import matplotlib.pyplot as plt

tim = timer.Timer()

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
,  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Setting up files, paths and selections

In [ ]:
model = '3'
directory = f'/home/maria/Project/ROS1_62IC+KIN_2/resultsmodel{model}IC62KIN_2/'
base = f'model{model}IC62KIN_2-MD.part0001.'
tpr = f'{directory}{base}gro'
xtc = f'{directory}{base}xtc' 
trr_file = f'{directory}{base}trr'
pdb = f'corrected_xtc/trial/model{model}ROS1_2.pdb'

fig_path = f'/home/joyce/Project/Forces/Figures/ROS1_exp2/model{model}ROS1figs/'
tp = 0

u = mda.Universe(tpr,xtc)
ref = mda.Universe(pdb)
protein = u.select_atoms('protein')

rALK = protein.resids + 1060
rROS1 = protein.resids + 1883
resid_numbers = rROS1 #Specify which residue numbers should be in the RMSF plot

CA = np.isin(protein.ix, protein.select_atoms('name CA').ix)
BB = np.isin(protein.ix, protein.select_atoms('backbone').ix)
BB[0] = False
BODY = np.isin(protein.resids , np.arange(19, 148))

# Reading coordinates from the mda universe into an array

In [ ]:
def uni2arr(u, selection):
    '''
    This function reads the coordinates from an MDAnalysis universe into a numpy array.
    '''
    X = np.zeros((len(u.trajectory), len(selection), 3))
    for ix, frame in enumerate(u.trajectory):
        X[ix] = selection.positions
    return X

# Reading the forces into a numpy array

In [ ]:
def trr2arr(trr_file, selection):
    '''
    This function reads a force data from a trr file using the trr module and saves it into a numpy array.
    '''
    T = trr.TRR(trr_file, selection.ix)
    F = T.forces
    return F

# Correcting for periodic boundary condition (PBC)

In [ ]:
def dim2pbc(arr: np.ndarray) -> np.ndarray:
    '''
    Convert unit cell definition from PDB CRYST1 format to lattice definition.
    '''
    lengths = arr[:, :3]
    angles = arr[:, 3:] * (np.pi / 180)

    cosa = np.cos(angles)
    sing = np.sin(angles[:, 2])    
    
    pbc = np.zeros((len(arr), 9))
    pbc[:, 0] = lengths[:, 0]
    pbc[:, 3] = lengths[:, 1] * cosa[:, 2]
    pbc[:, 4] = lengths[:, 1] * sing    
    pbc[:, 6] = lengths[:, 2] * cosa[:, 1]
    pbc[:, 7] = lengths[:, 2] * (cosa[:, 0] - cosa[:, 1] * cosa[:, 2]) / sing
    pbc[:, 8] = (lengths[:, 2] ** 2 - (pbc[:, 6:8] ** 2).sum(axis=1)) ** 0.5
    
    return pbc.reshape((-1, 3, 3))

In [ ]:
def boxdim(u, selection):
    '''
    Saving and processing the box dimensions over the entire directory using the dim2pbc function.
    '''
    B = np.empty((len(u.trajectory), 6))
    for idx, f in enumerate(u.trajectory):
        B[idx] = selection.dimensions.copy()
    B = dim2pbc(B)
    return B

In [ ]:
def nojump(X, B, ref=None):
    """
    Remove periodic boundary condition (PBC) jumps from a molecular dynamics trajectory.

    This function unwraps the trajectory `X` using the provided simulation box definitions `B`,
    correcting for discontinuities caused by particles crossing periodic boundaries. The result
    is a continuous trajectory in real space. The first frame or a user-defined reference frame
    is used as the starting point for unwrapping.

    Parameters
    ----------
    X : ndarray, shape (n_frames, n_atoms, 3)
        Molecular dynamics trajectory positions in Cartesian coordinates.
    B : ndarray, shape (n_frames, 3, 3)
        Simulation box matrices corresponding to each frame in `X`.
    ref : ndarray, shape (n_atoms, 3), optional
        Reference frame for unwrapping. If None, the first frame of `X` is used.

    Returns
    -------
    X_unwrapped : ndarray, shape (n_frames, n_atoms, 3)
        The unwrapped trajectory with periodic jumps removed and positions expressed
        in real Cartesian space.

    Notes
    -----
    - The trajectory is converted to fractional coordinates relative to the box,
      corrected for jumps, and then converted back to Cartesian coordinates.
    - This function assumes orthorhombic or triclinic boxes defined per-frame in `B`.
    """
    if ref is None:
        ref = X[0]
    ref = ref @ np.linalg.inv(B[0])
    
    X = X @ np.linalg.inv(B)
    X[1:] = np.diff(X, axis=0) + 0.5
    X[0] -= ref - 0.5
    X -= np.floor(X)
    X[0] += ref
    X = np.cumsum(X - 0.5, axis=0) @ B

    return X

# Preparing the reference frame for the RMSD

In [ ]:
def refplot(X):
    '''
    Plotting the reference structure for the analysis, checking for artifacts.
    '''
    ref = X[0]
    for k in [0, 1], [0, 2], [1, 2]:
        plt.scatter(*ref[:, k].T, c=-np.arange(len(ref)), cmap='turbo')
        plt.gca().set_aspect('equal')
        k = f'{k[0]}_{k[1]}'
        tp = f'rmsd{k}structure'
        name = f'{fig_path}/{tp}model_{model}.png'
        plt.savefig(name, dpi = 300, bbox_inches='tight')
        print(f' Reference structure from dimension {k}')
        plt.show()

# Deriving the RMSD

In [ ]:
def rmsd_plot(X):
    '''
    Calculating the RMSD over time and plotting it.
    '''
    X -= X.mean(axis=1, keepdims=True)
    ref = X[0]
    rgyr = (X ** 2).sum(axis=(1, 2)) / len(ref)
    
    U, L, V = np.linalg.svd(ref.T @ X / len(ref))
    X = X @ (U @ V).transpose((0, 2, 1))
    refrmsd = np.sqrt(np.maximum(0, rgyr[0] + rgyr - 2 * L.sum(axis=-1))) / 10
    
    plt.plot(refrmsd, color = 'magenta')
    plt.xlabel('time in ps')
    plt.ylabel('RMSD')
    
    tp = 'rmsd'
    name = f'{fig_path}/{tp}model_{model}.png'
    plt.savefig(name, dpi = 300, bbox_inches ='tight')
    print('RMSD over time')
    plt.show()

# Calculation of the RMSF

In [ ]:
def rmsf_plot(X):
    '''
    Calculating the RMSF per residue and plotting it.
    '''
    X -= X.mean(axis=1, keepdims=True)
    nframes = X.shape[0]
    means = X.mean(axis=0, keepdims=True) 
    rmsf = (((X - means) ** 2).sum(axis=(0, 2))/nframes ) ** 0.5 
    resrmsf = []
    for i in range(1, (max(resid_numbers)+1)):
        resrmsf.append((rmsf[resid_numbers == i] **2).mean() **0.5)

    plt.plot(resrmsf, color = 'magenta')
    plt.xlabel('residue number')
    plt.ylabel('RMSF')
    tp = 'rmsf'
    name = f'{fig_path}/{tp}model_{model}.png'
    plt.savefig(name, dpi = 300, bbox_inches ='tight')
    print('RMSF per residue')
    plt.show()        

# Principal component analysis for coordinates

In [ ]:
def aligned_to_ref_coords(X, split_data):
    '''
    Aligning all the frames to the reference frame based on the coordinate data, which is set to the first with X[0},
    plotting it and splitting the data in a way that is preferred and saving it into variables P and R.
    '''
    nframes, natoms, ndim = X.shape
    X -= X.mean(axis=1, keepdims=True)
    sele = BB & BODY
    fit = sele
    ref = X[0]

    rctl = ref[fit] - ref[fit].mean(axis=0)
    means = X[ :, fit].mean(axis=1, keepdims=True)
    U, L, V = np.linalg.svd((rctl.T / (3 * len(rctl))) @ X[ :, fit])
    X = (X - means) @ (U @ V).transpose((0, 2, 1))

    mean = X[ :, sele].mean(axis=0)
    P, R = np.split((X[ :, sele] - mean), [split_data], axis=0) 
    P = P.reshape((-1, mean.size))
    R = R.reshape((-1, mean.size))

    plt.hist2d(*X[:, sele, 1:].reshape((-1, 2)).T, bins=256)
    plt.gca().set_aspect('equal')
    tp = 'alignedtoref_coordinates'
    name = f'{fig_path}/{tp}model_{model}.png'
    plt.savefig(name, dpi = 300, bbox_inches='tight')
    print('Alignment of all structures onto the reference structure for coordinate data')
    plt.show()

    return P , R, mean

In [ ]:
def scree_plot(R, var):   
    '''
    Determining the eigenvalues and eigenvectors for the last subset of the split trajectory (R),
    plotting the eigenvalues into a scree plot and printing the amount of variance the principal components cover. 
    '''
    evals, loadings = np.linalg.eigh(R.T @ (R / len(R)))
    loadings = loadings[:, ::-1]

    plt.scatter(np.arange(1, 26), evals[-1:-26:-1])
    plt.ylim((-1, max(1.1*evals)))
    tp = f'screeplot_{var}'
    name = f'{fig_path}/{tp}model_{model}.png'
    plt.savefig(name, dpi = 300, bbox_inches='tight')
    print(f'Scree plot for {var} PCs')
    plt.show()
    print(np.round(np.cumsum(evals[-1:-26:-1]) / evals.sum(), 2))
    return loadings

In [ ]:
def pca_coords(P,R, loadings, mean):   
    '''
    Calculating the principal components for the trajectory based on the coordinate data and the eigenvectors,
    plotting the combination of each principal component
    '''
    ncomponents = 10
    scores = R @ loadings[:, :ncomponents]
    pscores = P @ loadings[:, :ncomponents]
    variances = scores.var
    scores /= len(mean) ** 0.5
    pscores /= len(mean) ** 0.5

    for k in [0,1], [0,2], [1,2], [0,3], [1,3], [2,3], [0,4], [1,4], [2,4], [3,4]:
        print(f'Components {k[0]} and {k[1]} for coordinates:')
        fig, ax = plt.subplots(figsize=(10, 10))
        xp = pscores[..., k][:,0]
        yp = pscores[..., k][:,1]
        x = scores[..., k][:,0]
        y = scores[..., k][:,1]
        plt.scatter(xp,yp, c='magenta') 
        plt.scatter(x,y, c='green') 
        plt.gca().set_aspect('equal')
        k = f'{k[0]}+{k[1]}_'
        tp = f'PC{k}plot_coordinates_'
        name = f'{fig_path}/{tp}model_{model}.png'
        plt.savefig(name, dpi = 300, bbox_inches='tight')
        plt.show()

# Principal component analysis for forces

In [ ]:
def aligned_to_ref_forces(X,F, split_data):
        '''
    Aligning all the frames to the reference frame based on the forces data, which is set to the first with X[0},
    plotting it and splitting the data in a way that is preferred and saving it into variables P and R.
    '''
    nframes, natoms, ndim = X.shape
    X -= X.mean(axis=1, keepdims=True)
    sele = BB & BODY
    fit = sele
    ref = X[0]

    rctl = ref[fit] - ref[fit].mean(axis=0)
    means = X[ :, fit].mean(axis=1, keepdims=True)
    U, L, V = np.linalg.svd((rctl.T / (3 * len(rctl))) @ X[ :, fit])
    X = (X - means) @ (U @ V).transpose((0, 2, 1))
    F = F @ (U @ V).transpose((0, 2, 1))

    mean = F[ :, sele].mean(axis=0)
    P, R = np.split((F[ :, sele] - mean), [split_data], axis=0) 
    P = P.reshape((-1, mean.size))
    R = R.reshape((-1, mean.size))

    plt.hist2d(*F[:, sele, 1:].reshape((-1, 2)).T, bins=256)
    plt.gca().set_aspect('equal')
    tp = 'alignedtoref_forces'
    name = f'{fig_path}/{tp}model_{model}.png'
    plt.savefig(name, dpi = 300, bbox_inches='tight')
    print('Alignment of all structures onto the reference structure for force data')
    plt.show()
    return P, R, mean

In [ ]:
def pca_forces(P,R,loadings, mean):
        '''
    Calculating the principal components for the trajectory based on the forces data and the eigenvectors,
    plotting the combination of each principal component
    '''
    ncomponents = 10

    scores = R @ loadings[:, :ncomponents]
    pscores = P @ loadings[:, :ncomponents]
    variances = scores.var
    scores /= len(mean) ** 0.5
    pscores /= len(mean) ** 0.5

    for k in [0,1], [0,2], [1,2], [0,3], [1,3], [2,3], [0,4], [1,4], [2,4], [3,4]:
        print(f'Components {k[0]} and {k[1]} for forces:')
        fig, ax = plt.subplots(figsize=(10, 10))
        xp = pscores[..., k][:,0]
        yp = pscores[..., k][:,1]
        x = scores[..., k][:,0]
        y = scores[..., k][:,1]
        plt.scatter(xp,yp, c='blue') 
        plt.scatter(x,y, c='orange') 
        plt.gca().set_aspect('equal')
        k = f'{k[0]}+{k[1]}_'
        tp = f'PC{k}plot_forces_'
        name = f'{fig_path}/{tp}model_{model}.png'
        plt.savefig(name, dpi = 300, bbox_inches='tight')
        plt.show()


In [ ]:
def full_analysis(u, trr_file, selection):
    '''
    Combining all functions necessary for the analysis based on a trajectory containing coordinate data
    as well as a trajectory containing forces data. 
    '''    
    with tim('The entire analysis'):
        X = uni2arr(u,selection)
        F = trr2arr(trr_file, selection)
        B = boxdim(u,protein)
        X = nojump(X,B, ref.atoms.positions)
        
        refplot(X)
        rmsd_plot(X)
        rmsf_plot(X)
        
        Pc, Rc, meanc = aligned_to_ref_coords(X, 50)
        loadingsc = scree_plot(Rc, 'coordinates')
        pca_coords(Pc,Rc, loadingsc, meanc)
        Pf, Rf, meanf = aligned_to_ref_forces(X, F, 50)
        loadingsf = scree_plot(Rf, 'forces')
        pca_forces(Pf, Rf, loadingsf, meanf)

In [ ]:
full_analysis(u, trr_file, protein)